1) Image size has to be taken care for each model (both in augmentation part and in model defining part)

2) With data augmentation it takes around 5 minutes for each epoch, for 64 dense layers and 0.4 dropout

For 10 epochs , train and val accuracies are 23.8 and 21


3) if else in data augmentation can speed up a bit ???

4) Testing on test dataset is not done

5) Using grid search, can be changed to bayes later


7) Batch size changed from 32 to 64

In [ ]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Wandb 
!pip install wandb --upgrade


     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 48.1 MB/s 
     |████████████████████████████████| 144 kB 53.1 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3337523e30528685f11acf8afb19102368daf5dc2b203a4f12de2fbd9bd9af39
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import tensorflow as tf
import numpy as np
import wandb

from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Dense,Dropout,Input,Flatten,MaxPooling2D,BatchNormalization

from wandb.keras import WandbCallback

In [ ]:

# GPU test

%tensorflow_version 2.x
import tensorflow as tf
import timeit

# We run this to warm up; see: https://stackoverflow.com/a/45067900
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error states that this notebook is not '
      'configured to use a GPU. Change this in the notebook Settings via the Runtime type or  '
      'command palette (cmd/ctrl-shift-P) or the Edit menu .\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.905026865000025
GPU (s):
0.038871170999982496
GPU speedup over CPU: 74x


In [ ]:
# train and test directory locations

train_dir = "/content/drive/MyDrive/inaturalist_12K/train"
val_dir = "/content/drive/MyDrive/inaturalist_12K/val"

isSweep = False 

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
isSweep = True 

sweepMethod ='bayes' #['grid','bayes','random']

if isSweep:

  sweepConfig = {'name': sweepMethod + '-PartB-fine-tuning-sweep',
                'method': sweepMethod,
                'metric' : {'name': 'val_acc', 
                            'goal': 'maximize'}
                }

  parameters = {
      
      'modelName'  : {'values': ['ResNet50','InceptionV3','Xception','InceptionResNetV2']},
      'denseSize'  : {'values': [64,128]},
      'num_epochs' : {'values': [4,8]},
      'dropOut'    : {'values': [0.2,0.4]},
      'batchNorm'  : {'values': [0,1]},
      'isDataAug'  : {'values': [0,1]},
      'optimizer'  : {'values': ['adam']},
  }

  sweepConfig['parameters'] = parameters
  sweepID = wandb.sweep(sweepConfig, project="cs6910_assignment2", entity="cs6910_assignment")


Create sweep with ID: 9sx5rb3i
Sweep URL: https://wandb.ai/cs6910_assignment/cs6910_assignment2/sweeps/9sx5rb3i


In [ ]:
def createDataset(imgHeight,imgWidth,isDataAug):

  imgSize = (imgHeight, imgWidth) 
  batchSize = 32
  if isDataAug:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    featurewise_center=False,  # set input mean to 0 over the dataset
                    samplewise_center=False,  # set each sample mean to 0
                    featurewise_std_normalization=False,  # divide inputs by std of the dataset
                    samplewise_std_normalization=False,  # divide each input by its std
                    zca_whitening=False,  # apply ZCA whitening
                    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                    horizontal_flip=True,  # randomly flip images
                    vertical_flip=False
                    )

    train_data = train_datagen.flow_from_directory(
            train_dir,
            subset='training',
            target_size=imgSize,
            batch_size=batchSize,
            class_mode='categorical',
            shuffle = True,
            seed = 123)

    val_data  = train_datagen.flow_from_directory(
            train_dir,
            subset='validation',
            target_size=imgSize,
            batch_size=batchSize,
            class_mode='categorical',
            shuffle = True,
            seed = 123)
    
  else:
    train_data = tf.keras.utils.image_dataset_from_directory(
                          directory = train_dir,
                          labels = 'inferred',  
                          label_mode = 'categorical',
                          color_mode = 'rgb',
                          batch_size = batchSize,
                          image_size = imgSize,
                          shuffle = True,
                          seed = 5,
                          validation_split = 0.2,
                          subset = 'training')

    val_data = tf.keras.utils.image_dataset_from_directory(
                          directory = train_dir,
                          labels = 'inferred',  
                          label_mode = 'categorical',
                          color_mode = 'rgb',
                          batch_size = batchSize,
                          image_size = imgSize,
                          shuffle = True,
                          seed = 5,
                          validation_split = 0.2,
                          subset = 'validation')


    # Retaining 25 percent of train and validation data and discarding the rest
    len_train, len_val = len(train_data), len(val_data)
    train_data = train_data.take(int(0.25*len_train))
    val_data = val_data.take(int(0.25*len_val))

  return train_data , val_data

In [ ]:
def train(config=False):
  # with wandb.init(config=config):
  with tf.device('/device:GPU:0'):

    #Dynamic values for sweeping
    #Hardcoded for testing 
    denseSize = 64
    optimizer = 'adam'
    num_epochs = 50
    isDataAug = 1
    batchNorm=1 
    dropOut =0.4
    modelName = 'Xception'

    if isSweep:
      config = sweepConfig
      if config:
        print("Running the Model with the dynamic parameters ")
        print(config)
        wandb.init(config=config)
        config = wandb.init().config
        wandb.run.name = 'model_{}_epochs_{}_denseSize_{}_dropOut_{}_batchNorm_{}_isDataAug_{}_Optimizer_{}'\
                          .format(config.modelName, config.num_epochs, config.denseSize, config.dropOut, config.batchNorm, config.isDataAug, config.optimizer)
        modelName  = config.modelName
        denseSize  = config.denseSize
        num_epochs = config.num_epochs
        dropOut    = config.dropOut
        batchNorm  = config.batchNorm
        isDataAug  = config.isDataAug
        optimizer  = config.optimizer


    if(modelName == "ResNet50"):
      imgHeight = 224
      imgWidth  = 224

      base_model = ResNet50(
        weights = 'imagenet', # loading the weights, pre trained on ImageNet
        input_shape = (imgHeight,imgWidth,3),
        include_top = False # Dense layers of the model are not included at the top
        )

    elif(modelName == "InceptionV3"):
      imgHeight = 299
      imgWidth  = 299

      base_model = InceptionV3(
        weights = 'imagenet', # loading the weights, pre trained on ImageNet
        input_shape = (imgHeight,imgWidth,3),
        include_top = False # Dense layers of the model are not included at the top
        )

    elif(modelName == "Xception"):
      imgHeight = 299
      imgWidth  = 299

      base_model = Xception(
        weights = 'imagenet', # loading the weights, pre trained on ImageNet
        input_shape = (imgHeight,imgWidth,3),
        include_top = False # Dense layers of the model are not included at the top
        )

    elif(modelName == "InceptionResNetV2"):
      imgHeight = 299
      imgWidth  = 299

      base_model = InceptionResNetV2(
        weights = 'imagenet', # loading the weights, pre trained on ImageNet
        input_shape = (imgHeight,imgWidth,3),
        include_top = False # Dense layers of the model are not included at the top
        )

      
    train_data , val_data = createDataset(imgHeight,imgWidth,isDataAug) 

    for layers in base_model.layers:
      layers.trainable = False

    model = tf.keras.Sequential([
      tf.keras.Input(shape=(imgHeight,imgWidth,3,)),
      base_model,
      Flatten(),
      Dense(denseSize,activation='relu'),
      ])
      
    if batchNorm == 1:
      model.add(BatchNormalization())

    model.add(Dropout(dropOut))
    model.add(Dense(denseSize, activation='relu'))
    model.add(Dropout(dropOut))
    model.add(Dense(10 ,activation='softmax'))

    model.compile(
        optimizer = optimizer,  
        # Loss function to minimize
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),#'categorical_crossentropy',
        # List of metrics to monitor
        metrics=['accuracy'],
        )

    hist = model.fit(
        train_data,
        epochs=num_epochs,
        validation_data=val_data,
        callbacks = [wandb.keras.WandbCallback()]
        )
  
    

In [ ]:
# isSweep=False
if isSweep:
  wandb.agent(sweepID, function=train)
else:
  train()

wandb: Agent Starting Run: 8609vy9d with config:
wandb: 	batchNorm: 1
wandb: 	denseSize: 64
wandb: 	dropOut: 0.4
wandb: 	isDataAug: 0
wandb: 	modelName: Xception
wandb: 	num_epochs: 4
wandb: 	optimizer: adam


Running the Model with the dynamic parameters 
{'name': 'bayes-PartB-fine-tuning-sweep', 'method': 'bayes', 'metric': {'name': 'val_acc', 'goal': 'maximize'}, 'parameters': {'modelName': {'values': ['ResNet50', 'InceptionV3', 'Xception', 'InceptionResNetV2']}, 'denseSize': {'values': [64, 128]}, 'num_epochs': {'values': [4, 8]}, 'dropOut': {'values': [0.2, 0.4]}, 'batchNorm': {'values': [0, 1]}, 'isDataAug': {'values': [0, 1]}, 'optimizer': {'values': ['adam']}}}


Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 9999 files belonging to 10 classes.
Using 1999 files for validation.
Epoch 1/4


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


62/62 [==============================] - 28s 372ms/step - loss: 2.6777 - accuracy: 0.1250 - val_loss: 2.3686 - val_accuracy: 0.1667 - _timestamp: 1648985518.0000 - _runtime: 33.0000
Epoch 2/4
62/62 [==============================] - 24s 358ms/step - loss: 2.3786 - accuracy: 0.1512 - val_loss: 2.2480 - val_accuracy: 0.1813 - _timestamp: 1648985543.0000 - _runtime: 58.0000
Epoch 3/4
62/62 [==============================] - 25s 376ms/step - loss: 2.3168 - accuracy: 0.1744 - val_loss: 2.1817 - val_accuracy: 0.2354 - _timestamp: 1648985568.0000 - _runtime: 83.0000
Epoch 4/4
62/62 [==============================] - 23s 350ms/step - loss: 2.2554 - accuracy: 0.1900 - val_loss: 2.1922 - val_accuracy: 0.1979 - _timestamp: 1648985608.0000 - _runtime: 123.0000



...

94781440/94765736 [==============================] - 1s 0us/step
Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 9999 files belonging to 10 classes.
Using 1999 files for validation.
Epoch 1/8
62/62 [==============================] - 23s 296ms/step - loss: 2.9734 - accuracy: 0.1119 - val_loss: 2.2991 - val_accuracy: 0.1146 - _timestamp: 1648986007.0000 - _runtime: 30.0000
Epoch 2/8
62/62 [==============================] - 18s 276ms/step - loss: 2.4434 - accuracy: 0.1225 - val_loss: 2.2392 - val_accuracy: 0.1375 - _timestamp: 1648986046.0000 - _runtime: 69.0000
Epoch 3/8
62/62 [==============================] - 19s 277ms/step - loss: 2.5096 - accuracy: 0.1280 - val_loss: 2.2307 - val_accuracy: 0.1458 - _timestamp: 1648986064.0000 - _runtime: 87.0000
Epoch 4/8
62/62 [==============================] - 18s 268ms/step - loss: 2.3294 - accuracy: 0.1361 - val_loss: 2.2726 - val_accuracy: 0.1208 - _timestamp: 1648986085.0000 - _runtime: 108.0000
Epoch 5/8
62/62 [=

accuracy,▁▂▂▃▅▇▆█
epoch,▁▂▃▄▅▆▇█
loss,█▄▄▃▂▂▂▁
val_accuracy,▁▃▃▁▆█▆█
val_loss,█▆▅▇▃▄▂▁
accuracy,0.20262
best_epoch,7
best_val_loss,2.11285
epoch,7
loss,2.11869
val_accuracy,0.20417


wandb: Agent Starting Run: 567u4s16 with config:
wandb: 	batchNorm: 0
wandb: 	denseSize: 128
wandb: 	dropOut: 0.4
wandb: 	isDataAug: 1
wandb: 	modelName: InceptionResNetV2
wandb: 	num_epochs: 4
wandb: 	optimizer: adam


Running the Model with the dynamic parameters 
{'name': 'bayes-PartB-fine-tuning-sweep', 'method': 'bayes', 'metric': {'name': 'val_acc', 'goal': 'maximize'}, 'parameters': {'modelName': {'values': ['ResNet50', 'InceptionV3', 'Xception', 'InceptionResNetV2']}, 'denseSize': {'values': [64, 128]}, 'num_epochs': {'values': [4, 8]}, 'dropOut': {'values': [0.2, 0.4]}, 'batchNorm': {'values': [0, 1]}, 'isDataAug': {'values': [0, 1]}, 'optimizer': {'values': ['adam']}}}


Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
Epoch 1/4
250/250 [==============================] - 349s 1s/step - loss: 2.4256 - accuracy: 0.2301 - val_loss: 1.7758 - val_accuracy: 0.3747 - _timestamp: 1648986542.0000 - _runtime: 360.0000
Epoch 2/4
163/250 [==================>...........] - ETA: 1:31 - loss: 1.9844 - accuracy: 0.2730